In [8]:
import pymongo
import pandas as pd

db = pymongo.MongoClient().stargeo

In [10]:
samples = pd.DataFrame(list(db.samples.find()))
print(samples.shape)
samples[:5]

(111893, 7)


,_id,accession,attributes,geo_id,platform,series,tags
0,55784189384e10cfdb6dc0a7,GSM13,{'Melanoma': 'Melanoma'},13,GPL7,GSE1,"[Age, Melanoma]"
1,55784189384e10cfdb6dc0a6,GSM12,{},12,GPL7,GSE1,"[Age, Melanoma]"
2,55784189384e10cfdb6dc0a5,GSM11,"{'Melanoma': 'Melanoma', 'Age': '54'}",11,GPL7,GSE1,"[Age, Melanoma]"
3,55784189384e10cfdb6dc0a8,GSM14,"{'Melanoma': 'Melanoma', 'Age': '45'}",14,GPL7,GSE1,"[Age, Melanoma]"
4,55784189384e10cfdb6dc0a9,GSM15,"{'Melanoma': 'Melanoma', 'Age': '43'}",15,GPL7,GSE1,"[Age, Melanoma]"


In [25]:
tags_set = set()
for tags in samples.tags:
    for t in tags:
        tags_set.add(t)
set(t.lower().replace('_non_tumour', '').replace('_tumour', '').replace('_tumor', '') for t in tags_set if ('control' not in t.lower() and 'ctrl' not in t.lower()))

{'acc ',
 'ad',
 'age',
 'all_cells',
 'als',
 'aml_tissue',
 'bc',
 'be_tissue',
 'bipolar_disorder',
 'cad',
 'cd',
 'celiac',
 'chagas',
 'ckd',
 'copd',
 'crc',
 'depression',
 'df',
 'dhf',
 'diabetic_nephropathy',
 'dorsolateral_prefrontal_cortex',
 'dss',
 'eac',
 'eac_non',
 'eczema',
 'escc',
 'female',
 'gb',
 'gb_yesno',
 'gist_tissue ',
 'hbv_alf',
 'hbv_hcc',
 'hcc',
 'hcc_non',
 'hd',
 'hearing_loss',
 'hiv',
 'hiv_eliteresponse',
 'hiv_nonresistant',
 'hiv_resistant',
 'hiv_stage1',
 'hiv_stage2',
 'hiv_stage3',
 'hiv_standardresponse',
 'hnscc',
 'hnscc_non',
 'ir',
 'lcc',
 'leishmaniasis',
 'lymphoblastoid_cell_line',
 'lymphoma',
 'major_depressive_disorder',
 'malaria',
 'male',
 'mb_gender',
 'mb_histology',
 'mb_originalsubtype',
 'mb_yn',
 'mcc',
 'melanoma',
 'mets',
 'mg',
 'mm',
 'ms',
 'no_current_tobacco_use',
 'nonsmoker',
 'nr',
 'nsclc',
 'obesity',
 'old',
 'onset_age',
 'orbitofrontal_cortex',
 'pac',
 'pancreatic_cancer',
 'pc',
 'pc_tissue',
 'periphe

In [33]:
# t = pd.DataFrame(data=list(tags_set), columns=['tag'])
# t['disease'] = ''
# t.to_csv('data/tags.csv', sep=',')GSE38246 

In [89]:
samples[samples.tags.map(lambda x: 'UC' in x)].groupby('series').first()

,_id,accession,attributes,geo_id,platform,tags
series,,,,,,
GSE37283,557882a5384e10cfdb6ec64f,GSM915451,{'Uc': 'UC'},915451,GPL13158,"[UC, CRC_Control]"


In [112]:
import numpy as np
t = pd.read_csv('data/tags.csv')
t.columns = ['id', 'tag', 'disease']
t['disease'] = t['disease'].map(lambda x: x.lower().replace("'", '').strip())
t['disease'] = t['disease'].map(lambda x: np.nan if x == '' else x)
t['disease'].value_counts().shape

(48,)

In [106]:
samples['tags'] = samples['tags'].map(lambda tags: set(t.strip() for t in tags))
samples[:10]

,_id,accession,attributes,geo_id,platform,series,tags
0,55784189384e10cfdb6dc0a7,GSM13,{'Melanoma': 'Melanoma'},13,GPL7,GSE1,"set([Melanoma, Age])"
1,55784189384e10cfdb6dc0a6,GSM12,{},12,GPL7,GSE1,"set([Melanoma, Age])"
2,55784189384e10cfdb6dc0a5,GSM11,"{'Melanoma': 'Melanoma', 'Age': '54'}",11,GPL7,GSE1,"set([Melanoma, Age])"
3,55784189384e10cfdb6dc0a8,GSM14,"{'Melanoma': 'Melanoma', 'Age': '45'}",14,GPL7,GSE1,"set([Melanoma, Age])"
4,55784189384e10cfdb6dc0a9,GSM15,"{'Melanoma': 'Melanoma', 'Age': '43'}",15,GPL7,GSE1,"set([Melanoma, Age])"
5,55784189384e10cfdb6dc0aa,GSM16,"{'Melanoma': 'Melanoma', 'Age': '43'}",16,GPL7,GSE1,"set([Melanoma, Age])"
6,55784189384e10cfdb6dc0ab,GSM17,"{'Melanoma': 'Melanoma', 'Age': '75'}",17,GPL7,GSE1,"set([Melanoma, Age])"
7,55784189384e10cfdb6dc0ac,GSM18,{},18,GPL7,GSE1,"set([Melanoma, Age])"
8,55784189384e10cfdb6dc0ad,GSM19,{},19,GPL7,GSE1,"set([Melanoma, Age])"
9,55784189384e10cfdb6dc0ae,GSM20,{},20,GPL7,GSE1,"set([Melanoma, Age])"


In [117]:
# t.set_index('tag', inplace=True)
t[:10]
tag_to_disease = {}
for tag, disease in t.set_index('tag').disease.dropna().items():
    tag_to_disease[tag.strip()] = disease.strip()


In [120]:
def to_disease(tags):
    return set(tag_to_disease[tag] for tag in tags if tag in tag_to_disease)
samples['disease'] = samples['tags'].map(to_disease)
samples[:10]

,_id,accession,attributes,geo_id,platform,series,tags,disease
0,55784189384e10cfdb6dc0a7,GSM13,{'Melanoma': 'Melanoma'},13,GPL7,GSE1,"set([Melanoma, Age])",set([melanoma])
1,55784189384e10cfdb6dc0a6,GSM12,{},12,GPL7,GSE1,"set([Melanoma, Age])",set([melanoma])
2,55784189384e10cfdb6dc0a5,GSM11,"{'Melanoma': 'Melanoma', 'Age': '54'}",11,GPL7,GSE1,"set([Melanoma, Age])",set([melanoma])
3,55784189384e10cfdb6dc0a8,GSM14,"{'Melanoma': 'Melanoma', 'Age': '45'}",14,GPL7,GSE1,"set([Melanoma, Age])",set([melanoma])
4,55784189384e10cfdb6dc0a9,GSM15,"{'Melanoma': 'Melanoma', 'Age': '43'}",15,GPL7,GSE1,"set([Melanoma, Age])",set([melanoma])
5,55784189384e10cfdb6dc0aa,GSM16,"{'Melanoma': 'Melanoma', 'Age': '43'}",16,GPL7,GSE1,"set([Melanoma, Age])",set([melanoma])
6,55784189384e10cfdb6dc0ab,GSM17,"{'Melanoma': 'Melanoma', 'Age': '75'}",17,GPL7,GSE1,"set([Melanoma, Age])",set([melanoma])
7,55784189384e10cfdb6dc0ac,GSM18,{},18,GPL7,GSE1,"set([Melanoma, Age])",set([melanoma])
8,55784189384e10cfdb6dc0ad,GSM19,{},19,GPL7,GSE1,"set([Melanoma, Age])",set([melanoma])
9,55784189384e10cfdb6dc0ae,GSM20,{},20,GPL7,GSE1,"set([Melanoma, Age])",set([melanoma])


In [134]:
def make_train_set(samples):
    samples = samples[samples['disease'].map(lambda x: len(x) > 0)]
    print('Good samples {}'.format(samples.shape))
    db = pymongo.MongoClient().scraper_meta
    
    series_accession = list(samples['series'].unique())
    samples_accession = list(samples['accession'].unique())
    series = pd.DataFrame(list(db.series.find({'accession': {'$in': series_accession}})))
    samples_c = pd.DataFrame(list(db.samples.find({'accession': {'$in': samples_accession}})))
    samples_c = samples_c[['accession', 'title', 'description', 'source_name']]
    samples_c.columns = ['sample_accession', 'sample_title',
       'sample_description', 'sample_source_name']
    
    series = series[['accession', 'summary', 'title', 'overall_design']]
    
    samples = samples[['accession', 'series', 'disease']]
    samples.columns = ['sample_accession', 'accession', 'disease']
    
    _t = pd.merge(samples, samples_c)
    _t = pd.merge(_t, series)
    _t['disease'] = _t['disease'].map(','.join)
    _t['tissue'] = np.nan
    _t['norm_or_tumour'] = np.nan
    
    _t = _t[['accession', 'summary', 'title', 'overall_design',
       'disease', 'tissue', 'sample_accession', 'sample_title',
       'sample_description', 'norm_or_tumour', 'sample_source_name']]
    return _t
#     train_set = pd.merge(samples, series, left_on='series')
out = make_train_set(samples)
out.to_csv('data/stargeo_train_set.csv', set=';')

Good samples (47401, 8)


In [124]:
ts = pd.read_csv('data/out_training_set_exp.csv', sep=';')
print(ts.columns)
ts[:5]

Index(['Unnamed: 0', 'accession', 'summary', 'title', 'overall_design',
       'disease', 'tissue', 'sample_accession', 'sample_title',
       'sample_description', 'norm_or_tumour', 'sample_source_name'],
      dtype='object')


,Unnamed: 0,accession,summary,title,overall_design,disease,tissue,sample_accession,sample_title,sample_description,norm_or_tumour,sample_source_name
0,0,GSE50081,Background: Patients with early stage non-smal...,Validation of a histology-independent prognost...,Expression profiling was performed on RNA from...,lung cancer,lung,GSM1001510,N-1,Gene expression data from fresh frozen normal ...,Norm,normal lung
1,1,GSE50081,Background: Patients with early stage non-smal...,Validation of a histology-independent prognost...,Expression profiling was performed on RNA from...,lung cancer,lung,GSM1001511,N-2,Gene expression data from fresh frozen normal ...,Norm,normal lung
2,2,GSE50081,Background: Patients with early stage non-smal...,Validation of a histology-independent prognost...,Expression profiling was performed on RNA from...,lung cancer,lung,GSM1001512,N-3,Gene expression data from fresh frozen normal ...,Norm,normal lung
3,3,GSE50081,Background: Patients with early stage non-smal...,Validation of a histology-independent prognost...,Expression profiling was performed on RNA from...,lung cancer,lung,GSM1001513,N-4,Gene expression data from fresh frozen normal ...,Norm,normal lung
4,4,GSE50081,Background: Patients with early stage non-smal...,Validation of a histology-independent prognost...,Expression profiling was performed on RNA from...,lung cancer,lung,GSM1001514,N-5,Gene expression data from fresh frozen normal ...,Norm,normal lung
